# Подготовка

##Подключение хранилища

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Установка зависимостей

In [2]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.27.post2 peft trl triton==3.1.0
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer openai-whisper soundfile librosa
!pip install --no-deps unsloth ffmpeg
!pip install flash-attn --no-build-isolation --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s 

## Сервисные функции

Функция выделения аудио из видео файла

In [3]:
import subprocess

def extract_audio(video_path):
  audio_path = "/content/drive/MyDrive/DP_outputs/extracted_audio.wav"

  subprocess.run([
      "ffmpeg", "-i", video_path, "-acodec", "pcm_s16le", "-ar", "16000", audio_path, "-y"
  ])

  print("Аудио успешно извлечено:", audio_path)
  return audio_path

Функция разбиения аудио на части

In [4]:
import librosa
import numpy as np
import soundfile as sf
import os

def split_audio(audio_path):
  # Load full audio file
  chunks_path="/content/drive/MyDrive/DP_outputs/chunks"
  os.mkdir(chunks_path)
  audio, sr = librosa.load(audio_path, sr=16000)  # Ensure 16kHz
  chunk_duration = 30  # Split into 30-second chunks
  chunk_samples = chunk_duration * sr  # Number of samples per chunk

  # Create chunks
  chunks = [audio[i : i + chunk_samples] for i in range(0, len(audio), chunk_samples)]

  # Save chunks as individual WAV files
  chunk_paths = []
  for i, chunk in enumerate(chunks):
      chunk_file = f"{chunks_path}/audio_chunk_{i}.wav"
      sf.write(chunk_file, chunk, sr)
      chunk_paths.append(chunk_file)

  print(f"Аудио разделено на {len(chunk_paths)} частей.")

  return chunk_paths

Функция транскрибации

In [5]:
import torch
import torch.nn as nn
import gc
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from tqdm import tqdm

class LoRALayer(nn.Module):
    """Basic LoRA implementation for Linear layers."""
    def __init__(self, base_layer, r=8, alpha=32, dropout=0.2):
        super().__init__()
        self.base_layer = base_layer  # Original layer
        self.r = r  # Rank
        self.scaling = alpha / r

        # LoRA A and B matrices
        self.lora_A = nn.Linear(base_layer.in_features, r, bias=False)
        self.lora_B = nn.Linear(r, base_layer.out_features, bias=False)

         # Dropout between A and B
        self.dropout = nn.Dropout(p=dropout)

        # Initialize LoRA weights
        nn.init.kaiming_uniform_(self.lora_A.weight)
        nn.init.zeros_(self.lora_B.weight)

    def forward(self, x):
        return self.base_layer(x) + self.lora_B(self.lora_A(x)) * self.scaling

def get_transcription(chunk_paths):

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model_path="/content/drive/MyDrive/DP_models/final/whisper-finetuned-ru"
  model_name="openai/whisper-small"

  # Загружаем процессор и базовую модель Whisper
  processor = WhisperProcessor.from_pretrained(model_name)
  whisper_model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

  # Принудительное использование русского языка для декодера
  whisper_model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

  # Отключаем dropout перед инференсом
  for name, module in whisper_model.named_modules():
      if isinstance(module, nn.Linear) and any(k in name for k in ["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"]):
          parent = whisper_model.get_submodule(".".join(name.split(".")[:-1]))
          lora_layer = LoRALayer(module, r=16, alpha=32, dropout=0.4).to(device)
          setattr(parent, name.split(".")[-1], lora_layer)

  # Загружаем веса LoRA и конвертируем их в FP16
  lora_weights = torch.load(model_path + "/whisper_lora_weights.pth", map_location=device)
  for k, v in lora_weights.items():
      lora_weights[k] = v.half()  # Преобразуем в FP16

  # Загружаем LoRA веса в модель
  missing_keys, unexpected_keys = whisper_model.load_state_dict(lora_weights, strict=False)
  print("LoRA веса загружены.")
  print(f"Пропущенные ключи: {missing_keys}")
  print(f"Неожиданные ключи: {unexpected_keys}")

  # Переводим в режим инференса и полностью отключаем dropout
  whisper_model.eval()
  for name, module in whisper_model.named_modules():
      if isinstance(module, nn.Dropout):
          module.p = 0.0  # Полностью отключаем dropout

  # Применяем torch.compile() для ускорения (если поддерживается)
  try:
      whisper_model = torch.compile(whisper_model)
      print("torch.compile() успешно применён для ускорения инференса!")
  except AttributeError:
      print("torch.compile() не поддерживается в данной версии PyTorch, пропускаем...")

  # Transcribe each chunk
  transcriptions = []

  # Assuming chunk_paths is a list of audio chunk file paths
  for chunk_file in tqdm(chunk_paths, desc="Обработка частей", unit="chunk"):
      # Load audio (must be 16kHz for Whisper)
      audio, _ = librosa.load(chunk_file, sr=16000)

      # Tokenize input for Whisper
      input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(device)

      # Transcribe
      with torch.no_grad():
          predicted_ids = whisper_model.generate(input_features)

      transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
      transcriptions.append(transcription)

  # Combine all transcriptions
  full_transcription = " ".join(transcriptions)

  del whisper_model
  del processor
  del transcriptions
  del transcription
  del chunk_file
  del audio
  del input_features

  # Delete all variables that might hold tensors
  for obj in gc.get_objects():
      try:
          if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
              del obj
      except:
          pass

  gc.collect()  # Force garbage collection
  torch.cuda.empty_cache()  # Free GPU memory
  torch.cuda.ipc_collect()  # Collect GPU shared memory

  return full_transcription

Функции обработки текста

In [7]:
import re
from deepmultilingualpunctuation import PunctuationModel
import re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

def clean_transcript(input_text):
    """Очищает транскрипцию и разбивает на осмысленные предложения."""

    # 1. Удаляем бессмысленные фразы и шум
    common_phrases = [
    r"\bсбер какие\b", r"\bджой какие\b", r"\bафина какие\b", r"\bтихо\b",
    r"\bкакие деньги\b", r"\bкакие вопросы\b", r"\bкакие виды\b",
    r"\bкакие расклады\b", r"\bкакие правила\b", r"\bкакие кредиты\b",
    r"\bкакие платежные\b", r"\bкакие планы\b", r"\bкакие условия\b",
    r"\bкакие виды у фильма\b", r"\bкакие виды у города\b",
    r"\bкакие виды у банка\b", r"\bкакие вопросы мне нужны\b",
    r"\bкакие поечару\b", r"\bкакие как там\b", r"\bкакие у города\b",
    r"\bкакие какие\b", r"\bкакие виды говорить\b", r"\bкакие что делать\b",
    r"\bпожалуйста\b", r"\bтихо\b", r"\bафина\b", r"\bджой\b",
    r"\bменьше часов по работе\b", r"\bвопросы у моей семьи\b",
    r"\bбудем оттуда\b", r"\bбудем менять тогда\b", r"\bпродолжилась там же\b",
    r"\bсчетом объема\b", r"\bкуда же вы\b", r"\bкуда идем\b",
    r"\bгромче если можно\b", r"\bкак называется\b", r"\bэто тоже важно\b",
    r"\bработаем не работаем\b", r"\bэто понятно здесь\b", r"\bчто нам нужно\b",
    r"\bпосмотреть афина\b", r"\bпосмотреть как бы\b", r"\bпосмотреть пробуем\b",
    r"\bкуда перейдем\b", r"\bну то есть\b", r"\bто есть\b", r"\bи уже от этого\b",
    r"\bвиды у фильма\b", r"\bвиды \b",r"\bафина\b", r"\bджой\b",r"\bсбер\b",
    r"\bсалют среднее расстояние между нептуном и солнцем\b", r"\bсалют\b"]

    for phrase in common_phrases:
        input_text = re.sub(phrase, "", input_text, flags=re.IGNORECASE)

    # 2. Убираем повторы слов (например, "финансист финансист" → "финансист")
    input_text = re.sub(r"\b(\w+)\s+\1\b", r"\1", input_text, flags=re.IGNORECASE)

    # 3. Убираем длинные повторяющиеся буквы (например, "ээээ" → "э")
    input_text = re.sub(r"([a-zA-Zа-яА-Я])\1{3,}", r"\1", input_text)

    # 4. Разбиваем текст на предложения
    sentences = sent_tokenize(input_text)

    # 5. Фильтруем слишком короткие и бессмысленные предложения
    sentences = [sent for sent in sentences if len(sent.split()) > 3]

    # 6. Убираем случайные цифры, стоящие отдельно (например, "два три 9" → "два три")
    sentences = [re.sub(r"\b\d+\b", "", sent) for sent in sentences]

    # 7. Объединяем обратно в текст
    cleaned_text = " ".join(sentences)

    return cleaned_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Функция выделения ключевой информации из транскрипции

In [33]:
import unsloth
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
import gc
import json
import re
import time
import ast
from tqdm import tqdm
from pydantic import BaseModel, ValidationError

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define structured output format using Pydantic
class MeetingSummary(BaseModel):
    Summarization: str
    Topics: list[str]
    Actions: list[str]
    Problems: list[str]
    Decisions: list[str]


# Шаблон JSON
REFERENCE_JSON = {
    "Summarization": "Brief meeting summary...",
    "Topics": ["Topic 1", "Topic 2"],
    "Actions": ["Action 1", "Action 2"],
    "Problems": ["Problem 1", "Problem 2"],
    "Decisions": ["Decision 1", "Decision 2"]
}

# Фиксированный промпт без транскрипта
PROMPT_TEMPLATE = """\
Analyze the following meeting transcript and extract the key points:
1. **Summarization** – a brief summary of the meeting.
2. **Topics** – a list of topics discussed.
3. **Decisions** – key decisions made.
4. **Problems** – challenges or issues identified.
5. **Actions** – planned or taken actions.

Return the output **STRICTLY in the following JSON format**:
{{
  "Summarization": "Brief meeting summary...",
  "Topics": ["Topic 1", "Topic 2"],
  "Actions": ["Action 1", "Action 2"],
  "Problems": ["Problem 1", "Problem 2"],
  "Decisions": ["Decision 1", "Decision 2"]
}}

Meeting transcript (in Russian):
{transcript}

**Return only a valid JSON response in Russian language.**
**Do not include explanations, introductions, or extra text.**
**If a category is missing, return an empty array [].**

### Response:
"""

def clean_json_fields(data):
    """Очищает JSON, исправляя списки, если они закодированы как строки."""
    for key in ["Topics", "Actions", "Problems", "Decisions"]:
        if isinstance(data.get(key), list):
            cleaned_list = []
            for item in data[key]:
                if isinstance(item, str) and item.startswith("["):
                    try:
                        cleaned_list.extend(ast.literal_eval(item))  # Разворачиваем вложенные списки
                    except (SyntaxError, ValueError):
                        cleaned_list.append(item)  # Если ошибка, оставляем как есть
                else:
                    cleaned_list.append(item)
            data[key] = cleaned_list  # Обновляем JSON-объект
    return data

def extract_valid_json(text):
    """Извлекает первый JSON, отличный от шаблона."""
    text = text.strip()

    # Убираем "### Response:" и "Response:", если они есть
    text = re.sub(r"^### Response:\s*", "", text)
    text = re.sub(r"^Response:\s*", "", text)

    # Находим **все** JSON-блоки в тексте
    json_matches = re.findall(r"\{[\s\S]*?\}", text)

    for json_text in json_matches:
        try:
            # Пробуем загрузить как JSON
            extracted_json = json.loads(json_text)
        except json.JSONDecodeError:
            try:
                extracted_json = ast.literal_eval(json_text)  # Пробуем как Python-словарь
            except (ValueError, SyntaxError):
                continue  # Если ошибка, пропускаем этот JSON

        # Чистим JSON от вложенных строковых списков
        extracted_json = clean_json_fields(extracted_json)

        # Если JSON **отличается от шаблона**, возвращаем его
        if extracted_json and extracted_json != REFERENCE_JSON:
            return extracted_json

    return None  # Если не найдено отличного JSON

# Функция разбивает только текст транскрипта
def split_transcript(tokenizer, transcript, max_tokens=3000, overlap=300):
    tokens = tokenizer.tokenize(transcript)
    chunks = []
    start = 0

    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk = tokenizer.convert_tokens_to_string(tokens[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap  # Overlapping context

    return chunks

# Функция для генерации ответа
def generate_response(model, tokenizer, chunk: str) -> str:
    prompt = PROMPT_TEMPLATE.format(transcript=chunk)  # Вставляем только транскрипт
    inputs = tokenizer([prompt], return_tensors="pt", truncation=True, max_length=4096).to(device)

    with torch.inference_mode():
        output_ids = model.generate(**inputs, max_new_tokens=200, do_sample=False)

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

# Функция обработки чанка
def summarize_chunk(model, tokenizer, chunk):
    response = generate_response(model, tokenizer, chunk)
    #print("Response:\n", response)

    json_data = extract_valid_json(response)
    #print("JSON:\n", json_data)
    return json_data if json_data else {}  # Если JSON не найден, вернем {}

def merge_chunks(chunks):
    """Объединяет JSON-резюме всех чанков в один итоговый JSON."""
    if not chunks:
        return {
            "Summarization": "",
            "Topics": [],
            "Actions": [],
            "Problems": [],
            "Decisions": []
        }  # Возвращаем пустую структуру, если нет чанков

    merged_summary = " ".join(chunk.get("Summarization", "") for chunk in chunks)

    merged_topics = list(set(topic for chunk in chunks for topic in chunk.get("Topics", [])))
    merged_actions = list(set(action for chunk in chunks for action in chunk.get("Actions", [])))
    merged_problems = list(set(problem for chunk in chunks for problem in chunk.get("Problems", [])))
    merged_decisions = list(set(decision for chunk in chunks for decision in chunk.get("Decisions", [])))

    return {
        "Summarization": merged_summary,
        "Topics": merged_topics,
        "Actions": merged_actions,
        "Problems": merged_problems,
        "Decisions": merged_decisions,
    }

# Главная функция обработки встречи
def extract_meeting_summary(model, tokenizer, input_text: str, retry_attempts=3):
    attempts = 0
    while attempts < retry_attempts:
        chunks = split_transcript(tokenizer, input_text)  # Разбиваем только транскрипт
        chunk_summaries = []

        # Обрабатываем каждый чанк отдельно
        for chunk in chunks:
            summary = summarize_chunk(model, tokenizer, chunk)
            if summary:  # Если есть данные, добавляем
                chunk_summaries.append(summary)

        #print(chunk_summaries)
        # Проверяем, есть ли успешные JSON-ответы
        if not chunk_summaries:
            print(f"Ошибка: все чанки пустые после {retry_attempts} попыток!")
            return {  # Если все чанки пустые, возвращаем пустую структуру
                        "Summarization": "",
                        "Topics": [],
                        "Actions": [],
                        "Problems": [],
                        "Decisions": []
                    }

        # Объединяем все чанки
        final_summary = merge_chunks(chunk_summaries)
        return final_summary

def get_meeting_info(input_text):
    # Очищаем транскрипт (если clean_transcript() нужна, добавь её)
    cleaned_text = input_text  # Удали это, если используешь clean_transcript(input_text)

    deepseek_model_path = "/content/drive/MyDrive/DP_models/final/unsloth_deepseek_r1_finetuned_16K/checkpoint-401"
    max_seq_length=16_384
    max_new_tokens=2000

    deepseek_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = deepseek_model_path,
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = True,
        device_map = 'auto',
        trust_remote_code=True
        )

    deepseek_model=FastLanguageModel.for_inference(deepseek_model)

    result = extract_meeting_summary(deepseek_model, tokenizer, cleaned_text, retry_attempts=3)

    del deepseek_model
    del tokenizer
    gc.collect()  # Force garbage collection
    torch.cuda.empty_cache()  # Free GPU memory
    torch.cuda.ipc_collect()  # Collect GPU shared memory

    return result

Форматирование вывода для чтения

In [34]:
import re

def format_transcription(text):
    # Разбиваем текст по знакам окончания предложений (. ! ?), оставляя их в тексте
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    # Выводим каждое предложение на новой строке
    formatted_text = "\n".join(sentences)
    return formatted_text

In [62]:
import ast

def clean_list(lst):
    """Очищает список от закодированных строковых списков, убирает дубликаты и лишние символы."""
    seen = set()
    cleaned_list = []

    for item in lst:
        if isinstance(item, str):
            item = item.strip().replace("«", "").replace("»", "")

            # Проверяем, является ли item строковым списком (например, '["a", "b"]')
            if item.startswith("[") and item.endswith("]"):
                try:
                    parsed_item = ast.literal_eval(item)  # Преобразуем строку в список
                    if isinstance(parsed_item, list):
                        for sub_item in parsed_item:
                            sub_item = str(sub_item).strip().replace("[", "").replace("]", "")
                            if sub_item and sub_item not in seen:
                                cleaned_list.append(sub_item)
                                seen.add(sub_item)
                        continue  # Пропускаем добавление исходной строки
                except (ValueError, SyntaxError):
                    pass  # Если ошибка, рассматриваем как обычную строку

        # Разбиваем по запятым, если в строке несколько пунктов
        if "," in item:
            sub_items = [x.strip() for x in item.split(",") if x.strip()]
            for sub_item in sub_items:
                if sub_item and sub_item not in seen:
                    cleaned_list.append(sub_item)
                    seen.add(sub_item)
            continue  # Не добавляем исходную строку повторно

        if item and item not in seen:
            cleaned_list.append(item)
            seen.add(item)

    return cleaned_list

def remove_duplicates(lst):
    """Удаляет дубликаты из списка, сохраняя порядок, с учётом пробелов и регистра."""
    seen = set()
    cleaned = []
    for x in lst:
        cleaned_x = x.strip().lower()  # Приводим к нижнему регистру, убираем пробелы
        if cleaned_x not in seen:
            seen.add(cleaned_x)
            cleaned.append(x.strip())  # Сохраняем оригинальный формат строки
    return cleaned

def clean_meeting_summary(meeting_info):
    """Очищает JSON-данные от дубликатов и вложенных списков."""

    # Очистка списков + удаление дубликатов
    for key in ["Topics", "Actions", "Problems", "Decisions"]:
        meeting_info[key] = remove_duplicates(clean_list(meeting_info.get(key, [])))

    # Очистка резюме (по предложениям) с учётом пробелов и регистра
    if "Summarization" in meeting_info:
        sentences = [s.strip() for s in meeting_info["Summarization"].split('.') if s.strip()]
        meeting_info["Summarization"] = '. '.join(remove_duplicates(sentences))

    return meeting_info

def format_meeting_info(meeting_info):
    """Форматирует JSON-данные в человекочитаемый текст."""
    summary_text = '\n'.join(meeting_info.get('Summarization', '').split('. '))
    formatted_text = f"**Резюме встречи:**\n{summary_text}\n\n"

    formatted_text += "**Темы обсуждения:**\n"
    topics = meeting_info.get('Topics', [])
    formatted_text += "\n".join(f"- {topic}" for topic in topics) + "\n\n" if topics else "Нет тем.\n\n"

    formatted_text += "**Действия:**\n"
    actions = meeting_info.get('Actions', [])
    formatted_text += "\n".join(f"- {action}" for action in actions) + "\n\n" if actions else "Нет действий.\n\n"

    formatted_text += "**Проблемы:**\n"
    problems = meeting_info.get('Problems', [])
    formatted_text += "\n".join(f"- {problem}" for problem in problems) + "\n\n" if problems else "Нет проблем.\n\n"

    formatted_text += "**Решения:**\n"
    decisions = meeting_info.get('Decisions', [])
    formatted_text += "\n".join(f"- {decision}" for decision in decisions) + "\n" if decisions else "Нет решений.\n"

    return formatted_text

# Получение транскрипции из видео и определение ключевой информации

In [65]:
video_path = "/content/drive/MyDrive/DP_datasets/GMT20240104_103040.mp4"

audio_path=extract_audio(video_path)

chunk_paths=split_audio(audio_path)

full_transcription=get_transcription(chunk_paths)

meeting_info = get_meeting_info(full_transcription)

cleaned_meeting_info = clean_meeting_summary(meeting_info)
formatted_text = format_meeting_info(cleaned_meeting_info)

print("\n-----------------------------------------------------------------")
print("Транскрипция встречи")
print("-----------------------------------------------------------------")
print(format_transcription(full_transcription))
print("-----------------------------------------------------------------")
print("Ключевые моменты встречи")
print("-----------------------------------------------------------------")
print(formatted_text)

Аудио успешно извлечено: /content/drive/MyDrive/DP_outputs/extracted_audio.wav


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/DP_outputs/chunks'